In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


## Read Data 

In [2]:
def read_data(path, include_label):
    def scan_file():
        with open(path) as in_file:
            next(in_file)  # skip header
            for l in in_file:
                yield l.strip().split('\t')
    if include_label:
        return [(qid, did, q, d, float(label)) for qid, q, _, _, did, d, label in scan_file()]
    else:
        return [(qid, did, q, d) for qid, q, _, _, did, d, _ in scan_file()]

train = read_data('data/WikiQA/WikiQACorpus/WikiQA-train.tsv', include_label=True)
predict  = read_data('data/WikiQA/WikiQACorpus/WikiQA-test.tsv', include_label=False)

## Pack Data

In [3]:
train_pack = mz.pack(train[:10000])
predict_pack = mz.pack(predict[:1000])

In [4]:
train_pack.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [5]:
train_pack.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [6]:
train_pack.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0.0
1,Q1,D1-1,0.0
2,Q1,D1-2,0.0
3,Q1,D1-3,1.0
4,Q1,D1-4,0.0


In [7]:
train_pack.frame[:].head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0.0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0.0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0.0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1.0
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0.0


In [8]:
train_pack.append_text_length(inplace=True)
train_pack.frame[:].head()

Processing length_right with len: 100%|██████████| 9526/9526 [00:00<00:00, 775793.95it/s]


,id_left,text_left,length_left,id_right,text_right,length_right,label
0,Q1,how are glacier caves formed?,29,D1-0,A partly submerged glacier cave on Perito More...,58,0.0
1,Q1,how are glacier caves formed?,29,D1-1,The ice facade is approximately 60 m high,41,0.0
2,Q1,how are glacier caves formed?,29,D1-2,Ice formations in the Titlis glacier cave,41,0.0
3,Q1,how are glacier caves formed?,29,D1-3,A glacier cave is a cave formed within the ice...,61,1.0
4,Q1,how are glacier caves formed?,29,D1-4,"Glacier caves are often called ice caves , but...",128,0.0


In [9]:
predict_pack.frame[:].head()

,id_left,text_left,id_right,text_right
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-0,African immigration to the United States refer...
1,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-1,The term African in the scope of this article ...
2,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-2,From the Immigration and Nationality Act of 19...
3,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-3,African immigrants in the United States come f...
4,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,D0-4,"They include people from different national, l..."


## Preprocess Data

In [10]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
preprocessor.fit(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 993/993 [00:00<00:00, 7565.60it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 9526/9526 [00:02<00:00, 4003.18it/s]
Fitting vocabulary unit.: 100%|██████████| 816312/816312 [00:00<00:00, 3083037.10it/s]


In [11]:
train_pack_processed = preprocessor.transform(train_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 993/993 [00:00<00:00, 5333.18it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 9526/9526 [00:03<00:00, 2993.12it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 108/108 [00:00<00:00, 5145.33it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 992/992 [00:00<00:00, 2895.21it/s]


## Generate Data

In [12]:
train_generator = mz.DataGenerator(train_pack_processed, batch_size=32)

In [13]:
first_batch = train_generator[0]
last_batch = train_generator[-1]
batch_slice = train_generator[2:5]
X, y = first_batch

In [14]:
X

{'id_left': array(['Q1174', 'Q1114', 'Q1166', 'Q341', 'Q475', 'Q660', 'Q313', 'Q325',
        'Q521', 'Q433', 'Q1387', 'Q873', 'Q332', 'Q745', 'Q621', 'Q433',
        'Q96', 'Q206', 'Q866', 'Q783', 'Q749', 'Q379', 'Q975', 'Q573',
        'Q42', 'Q1024', 'Q866', 'Q576', 'Q981', 'Q478', 'Q30', 'Q1036'],
       dtype='<U5'), 'text_left': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), 'length_left': array([34, 24, 81, 41, 30, 31, 40, 30, 33, 23, 28, 39, 51, 38, 23, 23, 43,
        45, 68, 33, 25, 31, 40, 41, 28, 25, 68, 40, 22, 33, 41, 24]), 'id_right': array(['D1127-2', 'D1072-15', 'D1120-4', 'D340-5', 'D468-4', 'D644-6',
        'D312-4', 'D324-11', 'D513-1', 'D428-14', 'D1324-1', 'D846-11',
        'D331-10', 'D723-1', 'D339-7', 'D428-11', 'D96-1', 'D206-5',
        'D839-15', 'D757-5', 'D

In [15]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
X, y = batch_slice
y.shape

(96,)

## Model Building

In [17]:
ranking_task = mz.tasks.Ranking()
ranking_task.metrics = [
    'mae', 'map', 'precision',
    mz.metrics.Precision(k=3),
    mz.metrics.DiscountedCumulativeGain(k=1),
    mz.metrics.DiscountedCumulativeGain(k=3),
    mz.metrics.DiscountedCumulativeGain(k=5),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=1),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5)
]
ranking_task.metrics

['mae',
 mean_average_precision(0),
 precision@1(0),
 precision@3(0),
 discounted_cumulative_gain@1(0),
 discounted_cumulative_gain@3(0),
 discounted_cumulative_gain@5(0),
 normalized_discounted_cumulative_gain@1(0),
 normalized_discounted_cumulative_gain@3(0),
 normalized_discounted_cumulative_gain@5(0)]

In [18]:
import keras.layers
from matchzoo import engine

class SimpleEmbeddingModel(engine.BaseModel):
    @classmethod
    def get_default_params(cls) -> engine.ParamTable:
        """:return: model default parameters."""
        params = super().get_default_params(with_embedding=True)
        params.add(engine.param.Param(
            name='num_dense_units',
            value=512,
            hyper_space=engine.hyper_spaces.quniform(low=32, high=1024),
            validator=lambda x: 0 < x < 2048
        ))
        return params

    def build(self):
        """Model structure."""
        x_in = [keras.layers.Input(name=name, shape=shape)
                for name, shape in zip(['text_left', 'text_right'],
                                       self._params['input_shapes'])]
        x = keras.layers.concatenate(x_in)
        x = keras.layers.embeddings.Embedding(*self._params['embedding_shape'], name='embedding')(x)
        x = keras.layers.Dense(self._params['num_dense_units'],
                               activation='relu')(x)
        x_out = self._make_output_layer()(x)
        self._backend = keras.models.Model(inputs=x_in, outputs=x_out)

In [19]:
model = SimpleEmbeddingModel()
model.params['task'] = ranking_task
model.params['embedding_shape'] = (len(preprocessor.context['term_index']) + 1, 300)
model.guess_and_fill_missing_params()
model.build()
model.compile()

In [20]:
vocab = preprocessor.context['term_index']
embedding_matrix = np.random.uniform(-0.2, 0.2, model.params['embedding_shape'])

In [21]:
model.load_embedding_matrix(embedding_matrix)

## Model Training

In [22]:
model.fit(*train_pack_processed.unpack(), epochs=2, batch_size=32)

ValueError: Error when checking input: expected text_left to have shape (30,) but got array with shape (7888,)

In [ ]:
model.fit_generator(train_generator, epochs=2, workers=4, use_multiprocessing=True)

### Training with Dynamic Data Generator

In [ ]:
pp_no_hashing = mz.preprocessors.DSSMPreprocessor(with_word_hashing=False)
train_pack_processed_no_hashing = pp_no_hashing.fit_transform(train_pack)
word_hashing_unit = mz.processor_units.WordHashingUnit(pp_no_hashing.context['term_index'])
dynamic_data_generator = mz.data_generator.DynamicDataGenerator(word_hashing_unit.transform,
                                                                train_pack_processed_no_hashing)
model.fit_generator(dynamic_data_generator, epochs=2, workers=4, use_multiprocessing=True)

## Model Predicting 

In [ ]:
model.predict(predict_pack_processed[:10].unpack()[0])

## Model Evaluating

In [ ]:
model.evaluate(*train_pack_processed.unpack())